In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns 

In [2]:
v1 = pd.read_csv("integrated2.csv")
for i in v1.columns:
    print(i)

Unnamed: 0
lat
lon
PSurf1
PSurf10
PSurf11
PSurf12
PSurf13
PSurf14
PSurf15
PSurf16
PSurf17
PSurf18
PSurf19
PSurf2
PSurf20
PSurf21
PSurf22
PSurf23
PSurf24
PSurf25
PSurf26
PSurf27
PSurf28
PSurf29
PSurf3
PSurf30
PSurf31
PSurf32
PSurf33
PSurf34
PSurf35
PSurf36
PSurf37
PSurf38
PSurf39
PSurf4
PSurf40
PSurf41
PSurf42
PSurf43
PSurf44
PSurf45
PSurf46
PSurf47
PSurf48
PSurf49
PSurf5
PSurf50
PSurf51
PSurf52
PSurf6
PSurf7
PSurf8
PSurf9
Qair1
Qair10
Qair11
Qair12
Qair13
Qair14
Qair15
Qair16
Qair17
Qair18
Qair19
Qair2
Qair20
Qair21
Qair22
Qair23
Qair24
Qair25
Qair26
Qair27
Qair28
Qair29
Qair3
Qair30
Qair31
Qair32
Qair33
Qair34
Qair35
Qair36
Qair37
Qair38
Qair39
Qair4
Qair40
Qair41
Qair42
Qair43
Qair44
Qair45
Qair46
Qair47
Qair48
Qair49
Qair5
Qair50
Qair51
Qair52
Qair6
Qair7
Qair8
Qair9
Rainf1
Rainf10
Rainf11
Rainf12
Rainf13
Rainf14
Rainf15
Rainf16
Rainf17
Rainf18
Rainf19
Rainf2
Rainf20
Rainf21
Rainf22
Rainf23
Rainf24
Rainf25
Rainf26
Rainf27
Rainf28
Rainf29
Rainf3
Rainf30
Rainf31
Rainf32
Rainf33
Rainf3

In [3]:
v1 = v1.drop(columns=['Unnamed: 0' , "geometry" , "CNT_FULLNAME"])

Since the goal is to build clusters and not extracting information from the dataset  we can remove the high correlated data from the dataframe like subSoil / topSoil correlated data 

In [5]:
v1 = v1.drop(columns=['sand % subsoil','silt% subsoil', 'clay % subsoil', 'pH water subsoil','N % subsoil',
        'BS % subsoil','CEC subsoil', 'CEC Clay subsoil','CaCO3 % subsoil',  'BD subsoil'])

We also drop the correlated top soil chimical components

In [6]:
v1 = v1.drop(columns= ['BS % topsoil','BD topsoil'])

In [7]:
v1

,lat,lon,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,-8.25,27.25,89931.291146,89935.560190,89857.437607,89719.336024,89919.630089,89795.942393,89822.866530,89845.084958,...,22.11,7.22,0.583,0.336,0.063,12.58,57.1,5.46,9.8,9.9
1,-8.25,27.75,88198.302755,88202.255054,88144.626482,88011.359357,88205.834726,88097.959524,88101.279536,88143.110607,...,19.09,7.87,0.501,0.288,0.060,9.98,57.8,18.23,10.3,10.8
2,-8.25,28.25,89235.961901,89234.871446,89175.960774,89039.490220,89237.955708,89145.462488,89148.886125,89189.423988,...,19.09,7.87,0.501,0.288,0.060,9.98,57.8,18.23,10.3,10.8
3,-8.25,28.75,89689.187172,89694.181381,89638.232685,89500.870935,89700.801429,89613.034661,89615.432875,89656.726643,...,24.58,7.39,0.678,0.405,0.113,10.87,56.7,8.54,10.3,10.8
4,-7.75,27.25,93016.242297,93018.701482,92912.807863,92766.109494,92984.016548,92840.870970,92899.681970,92893.402655,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,10.75,24.25,95072.465089,94959.076833,94828.393851,94597.915542,94820.357179,94638.193423,94533.842458,94701.787935,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
585,11.25,23.25,94380.135922,94266.675756,94140.533315,93925.996935,94145.756238,93967.888226,93894.973268,94048.788238,...,12.45,3.55,0.485,0.200,0.065,5.20,27.5,0.05,5.5,4.0
586,11.25,23.75,94442.190333,94333.525786,94204.438512,93982.367929,94202.245667,94021.829244,93923.329673,94090.795161,...,0.00,0.00,0.000,0.000,0.000,0.00,0.0,0.00,0.0,0.0
587,11.25,24.25,94815.380714,94705.219577,94574.246238,94340.447714,94563.048720,94380.986940,94259.847762,94441.106321,...,24.80,7.70,0.330,0.230,0.100,8.70,66.0,13.10,8.0,11.0


Normlize the data for better distance calculation

In [8]:
# usin the previous part normalization function 
def MinMax_Normalization(df):
    df_minmax = pd.DataFrame()
    for column in df.columns:
        if df[column].dtype == 'float64' and column not in ["lat","lon"]:
            bas = min(df[column])
            haut = max(df[column])
            gap = haut - bas
            df_minmax[column] = (df[column] - bas)/gap
    return df_minmax

v1 = MinMax_Normalization(v1)
v1

C:\Users\abdo7\AppData\Local\Temp\ipykernel_11656\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_minmax[column] = (df[column] - bas)/gap
C:\Users\abdo7\AppData\Local\Temp\ipykernel_11656\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_minmax[column] = (df[column] - bas)/gap
C:\Users\abdo7\AppData\Local\Temp\ipykernel_11656\1251721374.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,0.643969,0.650159,0.649107,0.650683,0.648146,0.649672,0.648948,0.647550,0.647135,0.647381,...,0.366788,0.749740,0.241808,0.452830,0.217241,0.318642,0.758300,0.162791,0.471154,0.61875
1,0.560761,0.566320,0.565992,0.567405,0.565297,0.567261,0.565956,0.565361,0.564443,0.565555,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
2,0.610584,0.616267,0.616038,0.617535,0.615192,0.618101,0.616457,0.615888,0.614997,0.616900,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
3,0.632345,0.638483,0.638470,0.640031,0.637567,0.640795,0.638948,0.638454,0.637617,0.640012,...,0.407764,0.767394,0.281211,0.545822,0.389655,0.275329,0.752988,0.254621,0.495192,0.67500
4,0.792091,0.799289,0.797369,0.799240,0.796287,0.797456,0.797271,0.794755,0.795127,0.795795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.890818,0.893144,0.890322,0.888556,0.885061,0.884688,0.876048,0.882083,0.881109,0.879824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
585,0.857577,0.859653,0.856944,0.855794,0.852449,0.852155,0.845250,0.850549,0.848879,0.849049,...,0.206536,0.368640,0.201161,0.269542,0.224138,0.131712,0.365206,0.001491,0.264423,0.25000
586,0.860556,0.862886,0.860045,0.858543,0.855180,0.854773,0.846617,0.852578,0.851415,0.850457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
587,0.878475,0.880865,0.877990,0.876002,0.872622,0.872205,0.862840,0.869495,0.868750,0.866518,...,0.411413,0.799585,0.136873,0.309973,0.344828,0.220365,0.876494,0.390578,0.384615,0.68750


In [9]:
v1.value_counts()

PSurf1    PSurf10   PSurf11   PSurf12   PSurf13   PSurf14   PSurf15   PSurf16   PSurf17   PSurf18   PSurf19   PSurf2    PSurf20   PSurf21   PSurf22   PSurf23   PSurf24   PSurf25   PSurf26   PSurf27   PSurf28   PSurf29   PSurf3    PSurf30   PSurf31   PSurf32   PSurf33   PSurf34   PSurf35   PSurf36   PSurf37   PSurf38   PSurf39   PSurf4    PSurf40   PSurf41   PSurf42   PSurf43   PSurf44   PSurf45   PSurf46   PSurf47   PSurf48   PSurf49   PSurf5    PSurf50   PSurf51   PSurf52   PSurf6    PSurf7    PSurf8    PSurf9    Qair1     Qair10    Qair11    Qair12    Qair13    Qair14    Qair15    Qair16    Qair17    Qair18    Qair19    Qair2     Qair20    Qair21    Qair22    Qair23    Qair24    Qair25    Qair26    Qair27    Qair28    Qair29    Qair3     Qair30    Qair31    Qair32    Qair33    Qair34    Qair35    Qair36    Qair37    Qair38    Qair39    Qair4     Qair40    Qair41    Qair42    Qair43    Qair44    Qair45    Qair46    Qair47    Qair48    Qair49    Qair5     Qair50    Qair51    Qair52    

In [10]:
v1  = v1.drop_duplicates()
v1

,PSurf1,PSurf10,PSurf11,PSurf12,PSurf13,PSurf14,PSurf15,PSurf16,PSurf17,PSurf18,...,clay % topsoil,pH water topsoil,OC % topsoil,OC % subsoil,N % topsoil,CEC topsoil,CEC clay topsoil,CaCO3 % topsoil,C/N topsoil,C/N subsoil
0,0.643969,0.650159,0.649107,0.650683,0.648146,0.649672,0.648948,0.647550,0.647135,0.647381,...,0.366788,0.749740,0.241808,0.452830,0.217241,0.318642,0.758300,0.162791,0.471154,0.61875
1,0.560761,0.566320,0.565992,0.567405,0.565297,0.567261,0.565956,0.565361,0.564443,0.565555,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
2,0.610584,0.616267,0.616038,0.617535,0.615192,0.618101,0.616457,0.615888,0.614997,0.616900,...,0.316689,0.817238,0.207798,0.388140,0.206897,0.252786,0.767596,0.543530,0.495192,0.67500
3,0.632345,0.638483,0.638470,0.640031,0.637567,0.640795,0.638948,0.638454,0.637617,0.640012,...,0.407764,0.767394,0.281211,0.545822,0.389655,0.275329,0.752988,0.254621,0.495192,0.67500
4,0.792091,0.799289,0.797369,0.799240,0.796287,0.797456,0.797271,0.794755,0.795127,0.795795,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,0.890818,0.893144,0.890322,0.888556,0.885061,0.884688,0.876048,0.882083,0.881109,0.879824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
585,0.857577,0.859653,0.856944,0.855794,0.852449,0.852155,0.845250,0.850549,0.848879,0.849049,...,0.206536,0.368640,0.201161,0.269542,0.224138,0.131712,0.365206,0.001491,0.264423,0.25000
586,0.860556,0.862886,0.860045,0.858543,0.855180,0.854773,0.846617,0.852578,0.851415,0.850457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
587,0.878475,0.880865,0.877990,0.876002,0.872622,0.872205,0.862840,0.869495,0.868750,0.866518,...,0.411413,0.799585,0.136873,0.309973,0.344828,0.220365,0.876494,0.390578,0.384615,0.68750


Since none of the algorithme we will use is influenced by the presence of outliers we w'ont handle them